In [19]:
# Modules to import

!pip install --user nipy
!pip install --user nilearn
!pip install --user seaborn
!pip install --user keras

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
# Main Imports

import nilearn

from nilearn.image import resample_to_img

import pylab as plt

import numpy as np
import nibabel as nb
import os
import glob
import random
import pandas as pd
import re

import seaborn as sns #added
sns.set(style="darkgrid") #added

from nilearn.image import mean_img #added
from nilearn.plotting import plot_anat #added

# Principal code parameters

In [21]:
# Local directory where the Data are mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

#CSV fileroot 
fileName = f"{rootDirectory}/{dataDir}/MGMT_Labels.csv"

SUBJECT_NAME_REPEATED=["BIANCHI","BOVE","PROIETTI"]

#for i in range(len(SUBJECT_NAME_REPEATED)):
#    f"{SUBJECT_NAME_REPEATED[i]}"
#    print(f"{SUBJECT_NAME_REPEATED[i]}")

PATIENT_REMOVED=['Mitchell','Pascal','Mgmt','Barontini','Bergnach','Bernola','Catalani','Colafrancesco','Colamartini','Coletta','Angeli','Darida','Maso','Federico','Fiucci','Ionta','Lupo','Martella','Masci','Medici','Monacelli','Musat','Palmieri','Pierini','Podagrosi','ProiettiM','Testa','Tiberi','Tomao','Veronesi','Zangari','Rufini','Farella','Array']

SEQUENCE_1= "rCBV_registered"
SEQUENCE_2= "rCBV"
#T1_registered , T1
#T2_registered , T2
#FLAIR_registered , FLAIR
#ADC_registered , ADC
#rCBV_registered , rCBV
#MPRAGEMDC
MaskPath = "SOLID"
#SOLID
#NECROSI
#T1ROI
#T2ROI


#reference dimensions
dim1=192
dim2=256
dim3=144


'''Model parameters'''

#percentage of training test
p=0.8

# Specify shape of convolution kernel
kernel_size = (3, 3)

# Specify number of output categories
n_classes = 2

# Specify number of filters per layer
filters = 16

#added block
nEpochs = 100  # Increase this value for better results (i.e., more training)

batch_size = 16   # Increasing this value might speed up fitting

activation='relu'

activation_Dense='softmax'

# optimizer
learning_rate = 1e-5
#adam = Adam(lr=learning_rate)
#sgd = SGD(lr=learning_rate)

loss='categorical_crossentropy'

metrics=['accuracy']


In [22]:
ls {datasetDir}/

ALESSANDRINI_GLAUCO/       FARELLA_PAOLO/           PANETTI/
ANGELONI_GIUSEPPINA/       FEDERICO_FRANCESCO/      PASCAL/
ASSANTO_MARIA/             FERRAZZA_RITA/           PASCARELLA_P/
ASTA_G/                    FERRI_M_B/               PASSARI/
Array_Labels_Def.csv*      FILIPPONI_QUINTINO/      PELUSO_A/
BAGNOLI_VINCENZO/          FIUCCI_A/                PENNICCHI_R/
BALZAMO_A/                 FLORIO_FRANCESCO_PAOLO/  PIERI/
BARONTINI_MARIA_GIOVANNA/  FRATINI_RITA/            PIERINI_CATERINA/
BATTISTA_DOMENICA/         GATTAMORTA_NATALINA/     PINEDA_MARIA_ASSUNTA/
BERGNACH_SILVANO/          GEGGI_GIULIO/            PISTOIA_CARLO/
BERNOLA_TERESA/            GENNARI_CRISTIANO/       PODAGROSI_TERESA/
BERTUZZI_LUISA/            GIACCHERINI_M/           PODDA_ANTONINO/
BEVILACQUA_RITA/           GIANFELICI_LUISA/        POMPEI_F/
BIANCHI_GIOVANNI/          GIOIA_COSMO_DAMIANO/     PRINCIPI_ANNA_MARIA/
BIANCHI_ORAZIO/            GIORDANO_STEFANIA/       PROIETTI_GIOVANNI/
BIANCHI_S/ 

# Survival Labels dataframe

In [23]:
#CSV fileroot 
#fileName = "/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/Array_Labels_Def.csv"

#csv file
df = pd.read_csv(fileName, sep=";", header=None)

#Dataframe columns title
df.columns = ["Subject", "Survival"]

#Subject column as index
df = df.set_index('Subject')

#in order to check the entire dataframe of labels
pd.set_option('display.max_rows', len(df))
print(df)

                Survival
Subject                 
Alessandrini           0
Angeloni               0
Assanto                0
Asta                   1
Bagnoli                0
Balzamo                1
Battista               0
Bertuzzi               0
Bevilacqua             1
BianchiG               0
BianchiO               0
BianchiS               1
Biavati                1
Boezi                  1
BoveA                  1
Cacace                 0
Caldaroni              1
Camacci                0
Camplese               0
Capezzone              1
Carulli                1
Carzedda               1
Cellini                1
Cherri                 0
Cimpureanu             1
Colazzo                1
Cosimi                 1
Costanzi               1
Crescenzi              0
BoveP                  0
Paoli                  0
Diaspro                1
Carlantonio            0
Lorenzo                1
Marco                  0
Massa                  0
Dobrisan               0
Droghei                0


# Functions

In [24]:
#NORMALIZE_NAME: the function takes the subject path basename and splits it according to the underscore
#in order to take the first string and use it as index in dataframe.
#If a string is repeated, the function adds the first letter of the second string after the underscore

def normalize_name(subject_path, add_name):
    #Components= []
    subject = os.path.basename(subject_path)
    subject = re.sub(r'^(DE|D|DI|LO|DEL)_', '', subject)
    components =  subject.split("_")
    if add_name[0]==components[0] or add_name[1]==components[0] or add_name[2]==components[0] :
            return components[0].title() + components[1][0].capitalize()
    else:
            return components[0].title()
        
    
#GET_SUBJECT_METADATA:the function recives, as input, the subject path and the list of name repeated already known
#the output are the path base name (SURNAME_NAME) and the normalized name
#add_name needs to contain the list of name repeated


def get_subject_metadata(subject_path, subjects_with_name=[]):
    dirname = os.path.basename(subject_path)
    return (dirname, normalize_name(subject_path, add_name=[name for name in subjects_with_name]))

In [25]:
'''' Get index positions of value in dataframe '''
def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value]) # Get bool dataframe with True at positions where the given value exists
    seriesObj = result.any() # Get list of columns that contains the value
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames: # Iterate over list of columns and fetch the rows indexes where value exists
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
# Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

# Dataframe folders construction

In [26]:
#Path and Subject columns
Df_Subjects_dirs = pd.DataFrame()
for subject_path in glob.glob(f"{datasetDir}/*"):
    subjects_dirs=[get_subject_metadata(subject_path , subjects_with_name=[SUBJECT_NAME for SUBJECT_NAME in SUBJECT_NAME_REPEATED])]
    print(subjects_dirs)
    df_subject_dirs = pd.DataFrame(subjects_dirs,columns=["Path", "Subject"])
    Df_Subjects_dirs=pd.concat([Df_Subjects_dirs,df_subject_dirs],ignore_index=True)


#in order to check the correct constucrion of 
pd.set_option('display.max_rows', len(Df_Subjects_dirs))
print(Df_Subjects_dirs)

#Subject as index
Df_Subjects_dirs = Df_Subjects_dirs.set_index('Subject')
pd.set_option('display.max_rows', len(Df_Subjects_dirs))

#in order to check the correct constucrion of Df_Subjects_dirs
print(Df_Subjects_dirs)

#Unnecessary strings removal with "drop" function
x=Df_Subjects_dirs
x.drop(index='Array', columns='Path')
# Delete rows with index label a & b    
modX= x.drop([PATIENT for PATIENT in PATIENT_REMOVED])
#modX["Path"]
modX = pd.DataFrame(modX)

#in order to check the correct construction of the dataframe
pd.set_option('display.max_rows', len(modX))
print(modX)

[('ALESSANDRINI_GLAUCO', 'Alessandrini')]
[('ANGELONI_GIUSEPPINA', 'Angeloni')]
[('Array_Labels_Def.csv', 'Array')]
[('ASSANTO_MARIA', 'Assanto')]
[('ASTA_G', 'Asta')]
[('BAGNOLI_VINCENZO', 'Bagnoli')]
[('BALZAMO_A', 'Balzamo')]
[('BARONTINI_MARIA_GIOVANNA', 'Barontini')]
[('BATTISTA_DOMENICA', 'Battista')]
[('BERGNACH_SILVANO', 'Bergnach')]
[('BERNOLA_TERESA', 'Bernola')]
[('BERTUZZI_LUISA', 'Bertuzzi')]
[('BEVILACQUA_RITA', 'Bevilacqua')]
[('BIANCHI_GIOVANNI', 'BianchiG')]
[('BIANCHI_ORAZIO', 'BianchiO')]
[('BIANCHI_S', 'BianchiS')]
[('BIAVATI_S', 'Biavati')]
[('BOEZI_MARIO', 'Boezi')]
[('BOVE_A', 'BoveA')]
[('CACACE_PAOLO', 'Cacace')]
[('CALDARONI_ANNA', 'Caldaroni')]
[('CAMACCI_FILIBERTO', 'Camacci')]
[('CAMPLESE_CANDEROLA', 'Camplese')]
[('CAPEZZONE', 'Capezzone')]
[('CARULLI_L', 'Carulli')]
[('CARZEDDA_PAOLO', 'Carzedda')]
[('CATALANI_F', 'Catalani')]
[('CELLINI_T', 'Cellini')]
[('CHERRI_M', 'Cherri')]
[('CIMPUREANU_N', 'Cimpureanu')]
[('COLAFRANCESCO_ROCCO', 'Colafrancesco')]
[(

# Join function between Path and Labels

In [27]:
result = modX.join(df, on='Subject')

#in order to check if all the dataframe is construced in the right way
pd.set_option('display.max_rows', len(result))
result   

,Path,Survival
Subject,,
Alessandrini,ALESSANDRINI_GLAUCO,0.0
Angeloni,ANGELONI_GIUSEPPINA,0.0
Assanto,ASSANTO_MARIA,0.0
Asta,ASTA_G,1.0
Bagnoli,BAGNOLI_VINCENZO,0.0
Balzamo,BALZAMO_A,1.0
Battista,BATTISTA_DOMENICA,0.0
Bertuzzi,BERTUZZI_LUISA,0.0
Bevilacqua,BEVILACQUA_RITA,1.0


# NAN removal

In [28]:
#it takes the indexes to which the NaN corresponds
NAN_index=result['Survival'].index[result['Survival'].apply(np.isnan)]

#it takes the number of the corresponding row as int
df_index=result.index.values.tolist()
int_index=[df_index.index(i) for i in NAN_index]
#int_index

#than it is possible to obtain the corresponding survival label of the original array, df in this case
label=[df.iloc[x]['Survival'] for x in int_index]
label

#iteration to replace the NaNs
for l in label:
    result['Survival'].fillna(l,inplace=True)
    
#result is the final Dataframe with "Subject" as index and the columns "Path" and "Survival"
pd.set_option('display.max_rows', len(result))
result

,Path,Survival
Subject,,
Alessandrini,ALESSANDRINI_GLAUCO,0.0
Angeloni,ANGELONI_GIUSEPPINA,0.0
Assanto,ASSANTO_MARIA,0.0
Asta,ASTA_G,1.0
Bagnoli,BAGNOLI_VINCENZO,0.0
Balzamo,BALZAMO_A,1.0
Battista,BATTISTA_DOMENICA,0.0
Bertuzzi,BERTUZZI_LUISA,0.0
Bevilacqua,BEVILACQUA_RITA,1.0


In [29]:
#import pickle
#result = pickle.load( open( "DataFrame.pickle", "rb" ) )

# Matrix construction

In [34]:
#Data loading
listOfElems=[]  #array that need to obtain the effective subjects with the sequence that we want to analyse 
Dim = []  #array that need to contain the dimension of each image in order to check the most frequent or the minimum one
Data = [] #array that contains information of the image after the extraction of data from the nibabel format
IMG=[] #array that contains the "file.nii.gz" information thanks to the nibabel module 
for Path in result["Path"]: #here starts the iteration on all the paths written in the column "Path" in result dataframe
    #print(Path)
    if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii"): #I need just the Paths that contain a certain sequence

        IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii") #nobabel module that allow the loading of nifty file
        DATA= IMG_reg.get_data() #get_data takes the information about scale of gray
        a = [DATA.shape]
        Dim.append(a) #allows the array construction
        Data.append(DATA)  #allows the array construction
        IMG.append(IMG_reg)  #allows the array construction
        

        IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii") #that's the file of the mask that limits information only on a certain region of the tumor
        ROI_DATA=IMG_roi.get_data()
        b=[ROI_DATA.shape]
        Dim.append(b)
        Data.append(ROI_DATA)
        IMG.append(IMG_roi)
        
        path=[f"{Path}"]
        listOfElems.append(path) #allows the array construction
        
    else: #anyway there are some patient with the sequence with different name but with the same information, so I can include them with the "else"
        if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii"):
            IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii")
            DATA= IMG_reg.get_data()
            a = [DATA.shape]
            Dim.append(a)
            Data.append(DATA)
            IMG.append(IMG_reg)
           
            IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii")
            ROI_DATA=IMG_roi.get_data()
            b=[ROI_DATA.shape]
            Dim.append(b)
            Data.append(ROI_DATA)
            IMG.append(IMG_roi)
            
            path=[f"{Path}"]
            listOfElems.append(path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_dat

In [35]:
len(listOfElems)

86

In [36]:
'''Labels Array Construction'''

#the following iteration allows the extraction of indexes corresponding to the paths selected by the preavious iterations
T1_Subject=[]
for i in range(len(listOfElems)):
    Pos=getIndexes(result, listOfElems[i][0])
    print(Pos[0][0])
    T1_Subject.append(Pos[0][0])  #the T1_Subject array contains these information

#the following iteration takes the corresponding Survival information on the result Dataframe built before    
T1_Subject_array=np.asarray(T1_Subject)

T1_Labels=[]
for i in range(len(T1_Subject_array)):
    lab = result.loc[T1_Subject_array[i],"Survival"]
    T1_Labels.append(int(lab)) #creates the list that contains these information

T1_Labels=np.asarray(T1_Labels) #that's the relative array
#T1_Labels

#the following iteration allows the construction of the final labels array with doubled information of each element of T1_Labels array
Label_Def=[]
for x in range(len(T1_Labels)):
    label_Def=[[T1_Labels[x]]*2]
    Label_Def.append(label_Def)
    
Label_Def=np.asarray(Label_Def)
Label_Def=np.ravel(Label_Def)
Label_Def #that's the final label array that can be used for the training

Alessandrini
Assanto
Battista
Bertuzzi
Bevilacqua
BianchiO
Boezi
BoveA
Cacace
Caldaroni
Camacci
Camplese
Capezzone
Carzedda
Cellini
Cimpureanu
BoveP
Santis
Diaspro
Carlatonio
Marco
Massa
Dobrisan
Angelo
Emery
Fabiani
Florio
Fratini
Gattamorta
Geggi
Gennari
Gianfelici
Gioia
Giordano
Inciti
Isoni
Labella
Landone
Liberati
Lioce
Longo
Bello
Lupi
Maiolini
Maragno
Marcolini
Mariani
Martinez
Micheli
Moscardini
Movia
Nerone
Orlandi
Pagliaroli
Pagnotta
Palma
Panetti
Passari
Pieri
Pistoia
Pompei
Principi
ProiettiG
Quacquarelli
Quattrociocchi
Ricci
Romito
Ronzo
Rosari
Ruscito
Rusnac
Russo
Sala
Saltarelli
Soloviy
Sterpa
Sykula
Tavernese
Tavolucci
Tempestini
Temporin
Teofili
Troscia
Vitulano
Zanatta
Zeppa


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0])

In [37]:
'''Working on dimensions'''

#in order to work on dimensions it's necessary the array construction (as predicted in previous blocks)
Dim=np.asarray(Dim)   
Data=np.asarray(Data)
IMG=np.asarray(IMG)

#in order to know the minimum dimensions
Min_value=np.amin(Dim, axis=1)
Min_value
Min=np.amin(Min_value, axis=0)
Min_value.shape[0]

not_in_index = [k for k in range(Min_value.shape[0]) if not np.all(Min_value[k] == (dim1, dim2, dim3))] #that's the
#construction of an array that contains the position not corresponding to the dimension researched [(192,256,144)
#in this case]

pos_1=np.where(Min_value[:,0]==dim1) #position with first dimension equal to 192
pos_2=np.where(Min_value[:,1]==dim2) #position with first dimension equal to 256
pos_3=np.where(Min_value[:,2]==dim3) #position with first dimension equal to 144
eq=np.intersect1d(pos_1,pos_2)  #that command in order to find the intersection between the pos_1 and pos_2
index_IMG=np.intersect1d(eq,pos_3) #intersection that gives the complememntary information of not_in_index

#Here is given a random position that corresponds to the dimension request
def_index=random.choice(index_IMG)
print(def_index)


55


In [38]:
'''RESAMPLE BLOCK'''

#The resample function is executed only on images without the dimension request, respect to a random image with dimension (192,256,144)
'''for i in not_in_index:
    Res=nilearn.image.resample_to_img(IMG[i], IMG[def_index])
    IMG[i]=Res
    Data[i]=IMG[i].get_data()'''

'for i in not_in_index:\n    Res=nilearn.image.resample_to_img(IMG[i], IMG[def_index])\n    IMG[i]=Res\n    Data[i]=IMG[i].get_data()'

In [39]:
#np.save('rCBV_Solid', Data)

 
Data=np.load('rCBV_Solid.npy',allow_pickle=True )

In [40]:
'''Reshaping of Input Matrix'''

Input_matrix=np.empty((len(Data),dim1,dim2,dim3)) #in order to generate an empty array with a fixed shape

for i in not_in_index:

    Input_matrix[i,:,:,:]=np.array(Data[i])




for i in index_IMG:

    Input_matrix[i,:,:,:]=np.array(Data[i])
    
    
#in order to check the correct construction    
Input_matrix.shape

#import pickle
#pickle.dump( Input_matrix, open( "Input_matrix.pickle", "wb" ) )

(204, 192, 256, 144)

# TRAINING

In [41]:
'''Modules needed'''

import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.optimizers import Adam, SGD

from keras import backend as K

Using TensorFlow backend.


In [42]:
labels=Label_Def
from keras.utils import to_categorical
labels = to_categorical(labels)

labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [43]:
'''## Create list of indices and shuffle them
N = Input_matrix.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)

#  Cut the dataset at 80% to create the training and test set
N_80p = int(p * N)
indices_train = indices[:N_80p]
indices_test = indices[N_80p:]

# Split the data into training and test sets
X_train = Input_matrix[indices_train, ...]
X_test = Input_matrix[indices_test, ...]

print(X_train.shape, X_test.shape)

labels=Label_Def

#Outcome variable block added
y_train = labels[indices_train] == 0
y_test = labels[indices_test] == 1

from keras.utils import to_categorical #Convert a class vector (integer) into binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train'''

'## Create list of indices and shuffle them\nN = Input_matrix.shape[0]\nindices = np.arange(N)\nnp.random.shuffle(indices)\n\n#  Cut the dataset at 80% to create the training and test set\nN_80p = int(p * N)\nindices_train = indices[:N_80p]\nindices_test = indices[N_80p:]\n\n# Split the data into training and test sets\nX_train = Input_matrix[indices_train, ...]\nX_test = Input_matrix[indices_test, ...]\n\nprint(X_train.shape, X_test.shape)\n\nlabels=Label_Def\n\n#Outcome variable block added\ny_train = labels[indices_train] == 0\ny_test = labels[indices_test] == 1\n\nfrom keras.utils import to_categorical #Convert a class vector (integer) into binary class matrix\ny_train = to_categorical(y_train)\ny_test = to_categorical(y_test)\n\ny_train'

In [44]:
'''Parameters setting'''

# Get shape of input data
data_shape = Data[0].shape

data_shape

(192, 256, 144)

# Model

In [4]:
#model block added
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 2, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 4, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# optimizer
learning_rate = 1e-5
adam = Adam(lr=learning_rate)
sgd = SGD(lr=learning_rate)

model.compile(loss='binary_crossentropy',
              optimizer=adam, # swap out for sgd 
              metrics=['accuracy','binary_crossentropy'])

model.summary()

NameError: name 'K' is not defined

In [1]:
batch_size = 16   # Increasing this value might speed up fitting

In [3]:
import tensorflow as tf

'''Model fitting'''
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# patient early stopping
#es = EarlyStopping(monitor='accuracy', mode='max', patience=100)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50)

%time fit = model.fit(Input_matrix, labels, epochs=500,callbacks=[callback],validation_split=0.2, shuffle=True,batch_size=batch_size)

NameError: name 'model' is not defined

In [47]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# Model plot

In [48]:
fig = plt.figure(figsize=(40, 25))
epoch = np.arange(500) + 1
fontsize = 30
plt.plot(epoch, fit.history['accuracy'], marker="o", linewidth=2,
         color="steelblue", label="train")
plt.plot(epoch, fit.history['val_accuracy'], marker="o", linewidth=2,
         color="red", label="test")
plt.plot(epoch, fit.history['loss'], marker="o", linewidth=2,
         color="orange", label="loss")
plt.xlabel('Epoch', fontsize=fontsize)
plt.ylabel('% Accuracy', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=30);


NameError: name 'fit' is not defined

<Figure size 2880x1800 with 0 Axes>